In [29]:
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.animation as animation
import random

### Complex Geometry with Stochastic Elements

Another method of smoothing waves is to make their propagation stochastic. Because the conditions for a wave to propagate are more relaxed, the wave edge is less well defined, and is slightly smoothed. Here, we've added a random chance for nodes to transition between states, even when they meet the prerequisites eg. a neighbor is on fire. The fire spread is seperated into 4 steps:

1. Catching on fire: when a neighbor is on fire, a grid space has a FIRE_SPREAD_CHANCE liklihood to catch on fire as well
2. Burning out: while on fire, a grid space has a BURNOUT_CHANCE liklihood to be extinguished and burnt
3. Recovering: if a grid space is burnt, but doesn't have any recovering neighbors, it has a REGROW_ALONE_CHANCE liklihood to begin recovering. If it has a recovering neighbor, it instead has a REGROW_NEIGHBOR_CHANCE liklihood of beginning to recover
4. Fully recovered: if a grid space is recovering, it has a FULLY_REGROW_CHANCE likelihood of becoming reigniteable

In [57]:
# These are parameters you can change! I personally think these look nice,
# but feel free to play with them

grid_width = 51
seed_hex = int((1 / 2) * grid_width ** 2)

# these variables are defined in the cell above, and 
# influence the rate at which the system progresses
#
# You can tweak these probabilities to create some really cool 
# effects. Try lowering BURNOUT_CHANCE or raising REGROW_ALONE_CHANCE
# to create a self exciting system!

FIRE_SPREAD_CHANCE = .5
BURNOUT_CHANCE = .6
REGROW_ALONE_CHANCE =.03
REGROW_NEIGHBOR_CHANCE = .1
FULLY_REGROW_CHANCE = .05

In [58]:
# Define states. These influence the color of each state.
# Larger numbers mean brighter colors

ON = 300 # on is burning 
BURNT = 200 # burnt is not actively recovering
GROWING= 100 # is recovering 
OFF = 0 # can be ignited

random.seed(1379)

# these variable are covered in depth in the Complex Geometry section
# in short terms, they represent the grid, and which
# hexagons are adjacent to one another
adj_dict = gen_hex_adj_dict(grid_width)
x, y = gen_hex_grid(grid_width)
hex_count = len(x)
hex_state_global = [OFF for i in range(hex_count)]


def transition(data):

    global hex_state_global
    hex_state = hex_state_global.copy()
        
    # this ugly hunk of logic just runs through
    # all the possible states of each hexagon, 
    # and updates its state accordingly
    for i in range(hex_count):
        curr_state = hex_state_global[i]
        if curr_state == ON: 
            for neighbor in adj_dict[i]:
                if hex_state[neighbor] == OFF and random.random() < FIRE_SPREAD_CHANCE:
                    hex_state[neighbor] = ON
            if random.random() < BURNOUT_CHANCE:
                hex_state[i] = BURNT
        elif curr_state == BURNT and random.random() < REGROW_ALONE_CHANCE:
            hex_state[i] = GROWING
        elif curr_state == GROWING:
            for neighbor in adj_dict[i]:
                if hex_state[neighbor] == BURNT and random.random() < REGROW_NEIGHBOR_CHANCE:
                    hex_state[neighbor] = GROWING
            if random.random() < FULLY_REGROW_CHANCE:
                hex_state[i] = OFF
        
    hex_state_global = hex_state
    
    # these two random states are here so that
    # the colors in the animation stay normalized.
    # They do properly affect their neighbors, they just 
    # don't show it in the animation
    hex_state[0] = OFF - 1
    hex_state[1] = ON + 1
    
    return ax.hexbin(x, y, C = hex_state, gridsize = (grid_width - 1, int(grid_width / 2)), cmap = "summer")

hex_state_global[seed_hex] = ON
fig, ax = plt.subplots(figsize = (6,6))
mat = ax.hexbin(x, y, C = hex_state_global, gridsize = (grid_width - 1, int(grid_width / 2)), cmap = "summer")
ani = animation.FuncAnimation(fig, transition, interval=50,
                              save_count=50)

plt.close(fig)

from IPython.display import HTML
HTML(ani.to_jshtml())